In [1]:
import torch
import numpy as np

from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision.transforms as transforms
import torchvision.datasets as datasets


def custom_loader(batch_size, shuffle_test=False, data_path='./dataset/preprocessed'):
    # Add the necessary transforms
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.225, 0.225, 0.225])
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # Adjust this if your images are a different size
        transforms.ToTensor(),
        normalize
    ])

    # Load your custom dataset
    train_dataset = datasets.ImageFolder(root=data_path + '/train', transform=transform)
    test_dataset = datasets.ImageFolder(root=data_path + '/test', transform=transform)

    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle_test, pin_memory=True)

    return train_loader, test_loader


# Function to load CIFAR10 dataset
def cifar_loader(batch_size, shuffle_test=False):
    # Normalization values for CIFAR10 dataset
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
    std=[0.225, 0.225, 0.225])
    # Loading training dataset with data augmentation techniques
    train_dataset = datasets.CIFAR10('./data', train=True, download=True,
    transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize
    ]))
    # Loading test dataset
    test_dataset = datasets.CIFAR10('./data', train=False,
    transform=transforms.Compose([
    transforms.ToTensor(),
    normalize
    ]))
    # Creating data loaders for training and testing
    train_loader = td.DataLoader(train_dataset, batch_size=batch_size,
    shuffle=True, pin_memory=True)
    test_loader = td.DataLoader(test_dataset, batch_size=batch_size,
    shuffle=shuffle_test, pin_memory=True)
    return train_loader, test_loader
# Hyperparameters and settings


# class DataloaderName(Dataset):
#     def __init__(self, inputprameters):
#
#         #codes
#
#     def __getitem__(self, index):
#
#         # code
#
#         return output
#
#     def __len__(self):
#         return self.__data.shape[0]


class MultiLayerFCNet(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()

        # self.fc1 = nn.Linear(input_size, hidden_size)
        # self.fc2 = nn.Linear(hidden_size, hidden_size)
        # self.fc3 = nn.Linear(hidden_size, output_size)

        self.layer1=nn.Conv2d(3,32,3,padding=1,stride=1)
        self.B1 = nn.BatchNorm2d(32)
        self.layer2 = nn.Conv2d(32, 32, 3, padding=1, stride=1)
        self.B2 = nn.BatchNorm2d(32)
        self.Maxpool=nn.MaxPool2d(2)
        self.layer3 = nn.Conv2d(32, 64, 3, padding=1, stride=1)
        self.B3 = nn.BatchNorm2d(64)
        self.layer4 = nn.Conv2d(64, 64, 3, padding=1, stride=1)
        self.B4 = nn.BatchNorm2d(64)
        self.fc = nn.Linear(64*8*8, output_size)

    def forward(self, x):
        # x = F.relu(self.fc1(x.view(x.size(0),-1)))
        # x = F.relu(self.fc2(x))
        # return F.log_softmax(self.fc3(x), dim=1)


        x = self.B1(F.leaky_relu(self.layer1(x)))
        x =  self.Maxpool(F.leaky_relu(self.layer2(x)))
        x=self.B2(x)
        x=self.B3(F.leaky_relu(self.layer3(x)))
        x = self.B4(self.Maxpool(F.leaky_relu(self.layer4(x))))

        return self.fc(x.view(x.size(0),-1))

if __name__ == '__main__':

    batch_size = 64
    test_batch_size = 64
    input_size = 3 * 32 * 32  # 3 channels, 32x32 image size
    hidden_size = 50  # Number of hidden units
    output_size = 4  # Number of output classes (CIFAR-10 has 10 classes)
    num_epochs = 10

    # train_loader, _ = cifar_loader(batch_size)
    # _, test_loader = cifar_loader(test_batch_size)
    train_loader, test_loader = custom_loader(batch_size, data_path='./dataset/preprocessed')
    # dataloader = DataLoader(dataset=IrisDataset('iris.data'),
    #                         batch_size=10,
    #                         shuffle=True)

    epochs = 50
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = MultiLayerFCNet(input_size, hidden_size, output_size)
    model = nn.DataParallel(model)
    model.to(device)
    #model.load_state_dict(torch.load('path'))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    BestACC=0
    for epoch in range(epochs):
        running_loss = 0
        for instances, labels in train_loader:
            optimizer.zero_grad()

            output = model(instances)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(running_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            allsamps=0
            rightPred=0

            for instances, labels in test_loader:

                output = model(instances)
                predictedClass=torch.max(output,1)
                allsamps+=output.size(0)
                rightPred+=(torch.max(output,1)[1]==labels).sum()


            ACC=rightPred/allsamps
            print("epoch=",epoch)
            print('Accuracy is=',ACC*100)
            if ACC>BestACC:
                BestACC=ACC
                # torch.save(model.state_dict())
                # torch.save(model.state_dict(), 'path')
        model.train()


3.6284128725528717
epoch= 0
Accuracy is= tensor(37.7535)
2.6236517594920263
epoch= 1
Accuracy is= tensor(31.2013)
2.098679847187466
epoch= 2
Accuracy is= tensor(35.5694)
1.5859713786178165
epoch= 3
Accuracy is= tensor(39.1576)
1.3756964604059856
epoch= 4
Accuracy is= tensor(39.3136)
1.155057892203331
epoch= 5
Accuracy is= tensor(42.9017)
1.0806762956910663
epoch= 6
Accuracy is= tensor(42.9017)
1.0351655897166994
epoch= 7
Accuracy is= tensor(44.3058)
0.9834960119591819
epoch= 8
Accuracy is= tensor(44.3058)
0.9215841641028722
epoch= 9
Accuracy is= tensor(43.5257)
0.8680444210767746
epoch= 10
Accuracy is= tensor(43.2137)
0.76985112660461
epoch= 11
Accuracy is= tensor(41.1856)
0.7146502393815253
epoch= 12
Accuracy is= tensor(42.7457)
0.6331641640928056
epoch= 13
Accuracy is= tensor(46.0218)
0.5256885579890676
epoch= 14
Accuracy is= tensor(41.8097)
0.4899928934044308
epoch= 15
Accuracy is= tensor(46.8019)
0.35179262566897607
epoch= 16
Accuracy is= tensor(41.6537)
0.26642698380682206
epoch= 